# Lab 3.3.5 Solutions: Production API

This notebook contains solutions to the exercises from the Production API task.

## Exercise 1: Add API Key Authentication

Implement API key authentication to protect your endpoint.

In [ ]:
# Solution: API Key Authentication

from fastapi import FastAPI, HTTPException, Depends, status
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from typing import Optional
import os

# Security scheme
security = HTTPBearer(auto_error=False)

# Valid API keys (in production, store these securely!)
VALID_API_KEYS = {
    "sk-test-key-12345": {"user": "test_user", "tier": "free"},
    "sk-prod-key-67890": {"user": "prod_user", "tier": "premium"},
}

# Get from environment variable for production
MASTER_KEY = os.getenv("API_MASTER_KEY", "")
if MASTER_KEY:
    VALID_API_KEYS[MASTER_KEY] = {"user": "admin", "tier": "unlimited"}


async def verify_api_key(
    credentials: Optional[HTTPAuthorizationCredentials] = Depends(security)
) -> dict:
    """
    Verify API key from Authorization header.
    
    Expected format: "Bearer sk-xxx"
    
    Returns:
        User info dict if valid
    
    Raises:
        HTTPException 401 if invalid or missing
    """
    if credentials is None:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Missing API key. Include 'Authorization: Bearer sk-xxx' header.",
            headers={"WWW-Authenticate": "Bearer"}
        )
    
    api_key = credentials.credentials
    
    if api_key not in VALID_API_KEYS:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Invalid API key.",
            headers={"WWW-Authenticate": "Bearer"}
        )
    
    return VALID_API_KEYS[api_key]


# Usage in endpoint:
app = FastAPI()

@app.post("/v1/chat/completions")
async def chat_completions(
    request: dict,
    user_info: dict = Depends(verify_api_key)  # Requires valid API key
):
    """Protected endpoint - requires valid API key."""
    return {
        "message": "Authenticated successfully!",
        "user": user_info["user"],
        "tier": user_info["tier"]
    }

print("✅ API Key Authentication Solution")
print("")
print("To use:")
print("  curl -H 'Authorization: Bearer sk-test-key-12345' http://localhost:8080/v1/chat/completions")

## Exercise 2: Add Request Logging to File

Log all requests to a JSON Lines file for later analysis.

In [ ]:
# Solution: Request Logging Middleware

import json
import time
from datetime import datetime
from pathlib import Path
from fastapi import FastAPI, Request
from starlette.middleware.base import BaseHTTPMiddleware
import asyncio

class RequestLoggingMiddleware(BaseHTTPMiddleware):
    """
    Middleware that logs all requests to a JSON Lines file.
    
    Each log entry contains:
    - timestamp
    - client_ip
    - method
    - path
    - status_code
    - latency_ms
    - user_agent
    """
    
    def __init__(self, app, log_file: str = "requests.jsonl"):
        super().__init__(app)
        self.log_file = Path(log_file)
        # Ensure log directory exists
        self.log_file.parent.mkdir(parents=True, exist_ok=True)
        # Buffer for async writing
        self._write_lock = asyncio.Lock()
    
    async def dispatch(self, request: Request, call_next):
        # Capture request info
        start_time = time.time()
        client_ip = request.client.host if request.client else "unknown"
        user_agent = request.headers.get("user-agent", "unknown")
        
        # Process request
        response = await call_next(request)
        
        # Calculate latency
        latency_ms = (time.time() - start_time) * 1000
        
        # Create log entry
        log_entry = {
            "timestamp": datetime.utcnow().isoformat() + "Z",
            "client_ip": client_ip,
            "method": request.method,
            "path": request.url.path,
            "query": str(request.url.query) if request.url.query else None,
            "status_code": response.status_code,
            "latency_ms": round(latency_ms, 2),
            "user_agent": user_agent[:100]  # Truncate long user agents
        }
        
        # Write to file asynchronously
        asyncio.create_task(self._write_log(log_entry))
        
        return response
    
    async def _write_log(self, entry: dict):
        """Write log entry to file."""
        async with self._write_lock:
            with open(self.log_file, "a") as f:
                f.write(json.dumps(entry) + "\n")


# Usage:
app = FastAPI()
app.add_middleware(RequestLoggingMiddleware, log_file="logs/requests.jsonl")

@app.get("/test")
async def test():
    return {"status": "ok"}

print("✅ Request Logging Middleware Solution")
print("")
print("Log file format (JSON Lines):")
example = {
    "timestamp": "2024-01-15T10:30:00.000Z",
    "client_ip": "192.168.1.100",
    "method": "POST",
    "path": "/v1/chat/completions",
    "status_code": 200,
    "latency_ms": 1523.45,
    "user_agent": "curl/7.68.0"
}
print(json.dumps(example, indent=2))

In [ ]:
# Bonus: Analyze logs

def analyze_logs(log_file: str) -> dict:
    """
    Analyze request logs from JSON Lines file.
    
    Returns:
        Summary statistics
    """
    from collections import Counter
    
    logs = []
    with open(log_file) as f:
        for line in f:
            logs.append(json.loads(line))
    
    if not logs:
        return {"error": "No logs found"}
    
    # Statistics
    latencies = [log["latency_ms"] for log in logs]
    status_codes = Counter(log["status_code"] for log in logs)
    paths = Counter(log["path"] for log in logs)
    
    return {
        "total_requests": len(logs),
        "time_range": {
            "first": logs[0]["timestamp"],
            "last": logs[-1]["timestamp"]
        },
        "latency": {
            "avg_ms": sum(latencies) / len(latencies),
            "min_ms": min(latencies),
            "max_ms": max(latencies),
            "p50_ms": sorted(latencies)[len(latencies) // 2],
            "p99_ms": sorted(latencies)[int(len(latencies) * 0.99)]
        },
        "status_codes": dict(status_codes),
        "top_paths": dict(paths.most_common(5)),
        "error_rate": sum(1 for log in logs if log["status_code"] >= 400) / len(logs)
    }

print("📊 Log Analysis Function")
print("")
print("Usage:")
print("  stats = analyze_logs('logs/requests.jsonl')")
print("  print(json.dumps(stats, indent=2))")

## Key Takeaways

1. **API Key Authentication**:
   - Use FastAPI's `Depends` for clean dependency injection
   - Store keys securely (environment variables, secrets manager)
   - Include user metadata (tier, permissions) with each key
   - Return helpful error messages

2. **Request Logging**:
   - Use middleware for cross-cutting concerns
   - JSON Lines format is easy to parse and analyze
   - Async file writing prevents blocking
   - Include key metrics: timestamp, latency, status

3. **Production Considerations**:
   - Rotate logs periodically
   - Consider centralized logging (ELK, Datadog)
   - Add request IDs for tracing
   - Monitor error rates and latency trends